In [41]:
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_ollama import ChatOllama
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import ChatPromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.output_parsers import StructuredOutputParser, ResponseSchema

In [43]:
import json
from langchain.schema import Document

documents = []

# ----------------------------
# 1. Proses file formasi CPNS
# ----------------------------
cpns_file = '/Users/muhammadzuamaalamin/Documents/risetmandiir/helpsekfix/data/data.json'

with open(cpns_file, 'r', encoding='utf-8') as f:
    raw_cpns = json.load(f)

# Pastikan berupa list
if isinstance(raw_cpns, dict):
    raw_cpns = [raw_cpns]
elif not isinstance(raw_cpns, list):
    raise ValueError("File formasi harus berupa object atau array of object.")

# ----------------------------
# 2. Konversi ke Document
# ----------------------------
for idx, item in enumerate(raw_cpns):
    # Gabungkan semua field penting ke dalam teks (content)
    # Format teks dibuat agar mudah dipahami oleh model RAG
    text_parts = [
        f"Jabatan: {item.get('jabatan', 'Tidak disebutkan')}",
        f"Instansi: {item.get('instansi', 'Tidak disebutkan')}",
        f"Unit Kerja: {item.get('unit_kerja', 'Tidak disebutkan')}",
        f"Penempatan: {item.get('penempatan', 'Tidak disebutkan')}",
    ]
    
    if 'catatan_penempatan' in item:
        text_parts.append(f"Catatan Penempatan: {item['catatan_penempatan']}")
    
    text_parts.extend([
        f"Jenis Formasi: {item.get('jenis_formasi', 'Tidak disebutkan')}",
        f"Khusus Disabilitas: {'Ya' if item.get('khusus_disabilitas', False) else 'Tidak'}",
        f"Penghasilan (juta): {item['penghasilan']['min']} - {item['penghasilan']['max']}",
        f"Jumlah Kebutuhan: {item.get('jumlah_kebutuhan', 0)}",
    ])

    # Format kualifikasi pendidikan (list)
    pendidikan = item.get('kualifikasi_pendidikan', [])
    if isinstance(pendidikan, str):
        pendidikan = [pendidikan]
    pendidikan_str = "; ".join(pendidikan)
    text_parts.append(f"Kualifikasi Pendidikan: {pendidikan_str}")

    if 'website_instansi' in item:
        text_parts.append(f"Website Instansi: {item['website_instansi']}")

    # Gabung jadi satu teks
    page_content = "\n".join(text_parts)

    # Metadata untuk filtering cepat di RAG
    metadata = {
        "jabatan": item.get("jabatan"),
        "instansi": item.get("instansi"),
        "jenis_formasi": item.get("jenis_formasi"),
        "khusus_disabilitas": item.get("khusus_disabilitas", False),
        "penempatan": item.get("penempatan"),
        "min_gaji": item["penghasilan"]["min"],
        "max_gaji": item["penghasilan"]["max"],
        "jumlah_kebutuhan": item.get("jumlah_kebutuhan", 0),
        "sumber_file": cpns_file,
        "index": idx
    }

    # Buat Document
    doc = Document(page_content=page_content, metadata=metadata)
    documents.append(doc)

In [44]:
documents

[Document(metadata={'jabatan': 'PENELITI AHLI MUDA', 'instansi': 'Badan Riset dan Inovasi Nasional', 'jenis_formasi': 'CPNS Lulusan Terbaik', 'khusus_disabilitas': True, 'penempatan': 'Jakarta', 'min_gaji': 7, 'max_gaji': 11, 'jumlah_kebutuhan': 75, 'sumber_file': '/Users/muhammadzuamaalamin/Documents/risetmandiir/helpsekfix/data/data.json', 'index': 0}, page_content='Jabatan: PENELITI AHLI MUDA\nInstansi: Badan Riset dan Inovasi Nasional\nUnit Kerja: Badan Riset dan Inovasi Nasional | SEKRETARIAT UTAMA\nPenempatan: Jakarta\nJenis Formasi: CPNS Lulusan Terbaik\nKhusus Disabilitas: Ya\nPenghasilan (juta): 7 - 11\nJumlah Kebutuhan: 75\nKualifikasi Pendidikan: S3 semua jurusan'),
 Document(metadata={'jabatan': 'PENELITI AHLI MUDA', 'instansi': 'Badan Riset dan Inovasi Nasional', 'jenis_formasi': 'CPNS Diaspora', 'khusus_disabilitas': True, 'penempatan': 'Jakarta', 'min_gaji': 7, 'max_gaji': 11, 'jumlah_kebutuhan': 125, 'sumber_file': '/Users/muhammadzuamaalamin/Documents/risetmandiir/help

In [45]:
## Embedding
# 3) Siapkan embedding (sekali saja)
embeddings = HuggingFaceEmbeddings(
    model_name="/Users/muhammadzuamaalamin/Documents/fintunellm/model/multilingual-e5-large-instruct",
    model_kwargs={"device": "cpu"}   # ganti "cuda" / "mps" bila tersedia dan ingin pakai GPU/MPS
)
# embeddings = HuggingFaceEmbeddings(model_name="/Users/muhammadzuamaalamin/Documents/fintunellm/model/bge-m3",
#                                    model_kwargs={"device": "cpu"})

In [46]:

# 4) Buat FAISS index dari CHUNKS (penting: gunakan chunks)
db = FAISS.from_documents(documents, embeddings)
db.save_local("faiss_index")
del db  # hapus dari memori

# nanti kalau butuh lagi
db = FAISS.load_local("faiss_index", embeddings, allow_dangerous_deserialization=True)


In [47]:
# 5) Pencarian mirip/kemiripan
query = "s1 hukum"
# Jika tersedia, gunakan method yang mengembalikan score
try:
    results_with_score = db.similarity_search_with_score(query, k=3)
    for idx, (doc, score) in enumerate(results_with_score, 1):
        print(f"Result {idx} — score={score:.4f}\n{doc.page_content[:800]}\n")
except AttributeError:
    # fallback bila method with_score tidak ada
    results = db.similarity_search(query, k=3)
    for idx, doc in enumerate(results, 1):
        print(f"Result {idx}\n{doc.page_content[:800]}\n")


Result 1 — score=0.3227
Jabatan: ARSIPARIS AHLI PERTAMA
Instansi: Kejaksaan Agung
Unit Kerja: Kejaksaan Agung
Penempatan: Nasional
Catatan Penempatan: Lokasi mengikuti satuan kerja yang dipilih di SSCASN (Kejaksaan Agung, Kejati, atau Kejari di seluruh Indonesia).
Jenis Formasi: CPNS Putra/putri Papua Dan Papua Barat
Khusus Disabilitas: Tidak
Penghasilan (juta): 8.54 - 10.53
Jumlah Kebutuhan: 17
Kualifikasi Pendidikan: D4 PENGELOLAAN ARSIP DAN REKAMAN INFORMASI; D4 MANAJEMEN REKOD DAN ARSIP; D4 KEARSIPAN; D4 KEARSIPAN DIGITAL; S1 ILMU LINGKUNGAN; S1 ANTROPOLOGI; S1 ADMINISTRASI NEGARA; S1 KEHUTANAN; S1 EKONOMI; S1 PENDIDIKAN AGAMA ISLAM; S1 PENDIDIKAN KEAGAMAAN KATOLIK; S1 PENDIDIKAN KEAGAMAAN BUDDHA; S1 ILMU KELAUTAN; S1 PENDIDIKAN KEAGAMAAN HINDU; S1 PENDIDIKAN KEAGAMAAN KRISTEN; S1 SEJARAH; S1 ILMU SEJARAH; S1

Result 2 — score=0.3230
Jabatan: ARSIPARIS AHLI PERTAMA
Instansi: Kejaksaan Agung
Unit Kerja: Kejaksaan Agung
Penempatan: Nasional
Catatan Penempatan: Lokasi mengikuti satuan

In [48]:
# Setup LLM dan memory
llm = ChatOllama(model="gemma3:4b")  # sedikit randomness boleh, tapi rendah
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# Buat retriever
retriever = db.as_retriever(search_kwargs={"k": 3})


In [49]:
from langchain.output_parsers import ResponseSchema, StructuredOutputParser
schemas = [
        ResponseSchema(
            name="jawaban",
            description="Jawaban lengkap, jelas, dan sesuai instruksi dalam bahasa Indonesia yang mudah dipahami."
        )
    ]
output_parser = StructuredOutputParser.from_response_schemas(schemas)
format_instructions = output_parser.get_format_instructions()


In [50]:
from langchain.prompts import ChatPromptTemplate
prompt_template = ChatPromptTemplate.from_template("""
Kamu adalah asisten resmi dari Badan Kepegawaian Negara (BKN) yang membantu masyarakat memahami informasi seputar Aparatur Sipil Negara (ASN), termasuk CPNS, PPPK, formasi, kualifikasi pendidikan, alur pendaftaran, dan regulasi terkait.

Instruksi:
1. Jawab PERTANYAAN PENGGUNA secara LENGKAP namun LANGSUNG KE INTI.
   - Jika tentang formasi: sebutkan jabatan, instansi, jumlah kebutuhan, kualifikasi, dan gaji (jika ada).
   - Jika tentang konsep (misal: “Apa itu CPNS?”): beri penjelasan sederhana dalam bahasa Indonesia.
2. “S-1 semua jurusan” berarti semua lulusan S-1 boleh mendaftar (berlaku juga untuk S-2/S-3).
3. Jika ada banyak formasi:
   - Tampilkan SEMUA yang relevan.
   - Setiap formasi di baris terpisah.
   - Urutkan dari jumlah kebutuhan terbanyak → sedikit, lalu penghasilan sebagai pembanding.
4. Gunakan HANYA informasi dalam KONTEKS. Jangan mengarang.
5. Jika konteks tidak cukup:  
   → Untuk topik ASN: "Maaf, saya tidak tahu jawaban pastinya berdasarkan dokumen yang tersedia."  
   → Di luar topik: "Maaf, saya tidak tahu. Saya hanya bisa menjawab pertanyaan seputar formasi CPNS, kualifikasi pendidikan, dan informasi rekrutmen ASN berdasarkan dokumen resmi."
6. Arahkan ke https://sscasn.bkn.go.id untuk info lebih lanjut.

Pertanyaan pengguna: {question}
Konteks tambahan dari dokumen:
{context}
{format_instructions}
""")

In [51]:

# Chain
chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    memory=memory,
    verbose=False,
    combine_docs_chain_kwargs={
        "prompt": prompt_template.partial(format_instructions=format_instructions)
    }
)

In [60]:
# Normalisasi query pengguna
query = "formasi yang penempatannya ada di banten formasi apa aja ?"
response = chain.invoke({"question": query})

# Parsing
try:
    if isinstance(response, dict) and "answer" in response:
        raw_answer = response["answer"]
    else:
        raw_answer = str(response)

    parsed = output_parser.parse(raw_answer)
    jawaban = parsed.get("jawaban", "Maaf, saya tidak dapat memberikan jawaban.")
except Exception as e:
    print("⚠️ Gagal parsing:", e)
    print("Raw output:", raw_answer)
    jawaban = "Maaf, terjadi kesalahan saat memproses jawaban."

print("\n==============================")
print("💬 Jawaban Asisten BKN:")
print(jawaban)
print("==============================\n")


💬 Jawaban Asisten BKN:
Berdasarkan data yang tersedia, saat ini tidak terdapat formasi CPNS dengan penempatan di Banten. Semua formasi CPNS yang terdaftar saat ini memiliki penempatan di Jakarta, yaitu:

*   **PENELITI AHLI MUDA**
    Instansi: Badan Riset dan Inovasi Nasional
    Unit Kerja: Badan Riset dan Inovasi Nasional | SEKRETARIAT UTAMA
    Penempatan: Jakarta
    Jenis Formasi: CPNS Penyandang Disabilitas
    Khusus Disabilitas: Ya
    Penghasilan (juta): 7 - 11
    Jumlah Kebutuhan: 10
    Kualifikasi Pendidikan: S3 semua jurusan

*   **PENELITI AHLI MUDA**
    Instansi: Badan Riset dan Inovasi Nasional
    Unit Kerja: Badan Riset dan Inovasi Nasional | SEKRETARIAT UTAMA
    Penempatan: Jakarta
    Jenis Formasi: CPNS Lulusan Terbaik
    Khusus Disabilitas: Ya
    Penghasilan (juta): 7 - 11
    Jumlah Kebutuhan: 75
    Kualifikasi Pendidikan: S3 semua jurusan

*   **PENELITI AHLI MUDA**
    Instansi: Badan Riset dan Inovasi Nasional
    Unit Kerja: Badan Riset dan Inovasi Na